# Subsetting summa input files by selecting a specific hruId
This is optional if you want to select a subset hru from the entire domain.
Needs to run `write_summa_files.ipynb` first to generate summa's inputs.

In [66]:
# packages are loaded
from summaflow import *

In [67]:
# inputs
# Set the folder path where the remapped .nc file is located for MESH (it can be any remapped nc file)
summa_input_path = '/scratch/mia725/SCRB/SUMMA/'
subset_output_path = '/scratch/mia725/SCRB/SUMMA-subset/'
hruId = [71034539] # required hruId for subsetting

# Start subsetting

In [6]:
if not os.path.isdir(subset_output_path):
    os.makedirs(subset_output_path)

In [75]:
forcing = xr.open_dataset(summa_input_path+'summa_forcing.nc')
# select a specific hru and save to subset folder
forcing.sel(hru=hruId).to_netcdf(subset_output_path+'summa_forcing.nc')


# replace T in the time unit with space

ncid = nc4.Dataset(subset_output_path + 'summa_forcing.nc', 'r+')

# Access the 'units' attribute and replace 'T' with a space
units_attribute = ncid['time'].units
units_attribute = units_attribute.replace('T', ' ')

# Update the 'units' attribute in the netCDF file
ncid['time'].setncattr('units', units_attribute)

# Close the netCDF file
ncid.close()

# write attribute nc file

In [74]:
attr = xr.open_dataset(summa_input_path+'summa_zLocalAttributes.nc')
# attr_subset = attr.sel(hru=hruId, gru=hruId)#.to_netcdf(subset_output_path+'summa_zLocalAttributes.nc')

# Create a dictionary to store the first value of each variable that lacks coordinates or dimensions
attr_subset = xr.Dataset()

for var_name in list(attr.keys()):
    dim_name = 'hru'
    if var_name == 'gruId':
        dim_name = 'gru'
    # Select the first value
    attr_subset[var_name] = xr.DataArray([attr[var_name].values[-1]], dims = (dim_name))  # Using item() to get the scalar value

# manually change certain values
attr_subset['hruId'][0] = attr_subset['gruId'][0] = attr_subset['hru2gruId'][0] = 71034539
attr_subset['HRUarea'][0] = attr['HRUarea'].values.sum()
attr_subset['contourLength'][0] = 42525.702
attr_subset.to_netcdf(subset_output_path+'summa_zLocalAttributes.nc')

# write ParamTrial file
This file only includes the hruId and the parameters are read from lookup tables.

In [51]:
parTrial = xr.open_dataset(summa_input_path+'summa_zParamTrial.nc')
parTrial.sel(hru=hruId).to_netcdf(subset_output_path+'summa_zParamTrial.nc')

# write intialCondition file

In [53]:
initCond = xr.open_dataset(summa_input_path+'summa_zInitialCond.nc')
initCond.sel(hru=hruId).to_netcdf(subset_output_path+'summa_zInitialCond.nc')

# write fileManager

In [54]:
write_summa_filemanager(subset_output_path, forcing)

# copy setting and static files

In [55]:
copy_summa_static_files(subset_output_path)